In [1]:
# debug cell
with open('simple_table.txt', 'r') as f:
    print(type(f))
    cnt = 2
    for i,line in enumerate(f):
        print(i, ':\t', line)
        cnt -= 1
        if cnt == 0:
            break

<class '_io.TextIOWrapper'>
0 :	 CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO:AF	UKB-b-6134:ES	UKB-b-6134:SE	UKB-b-6134:LP	UKB-b-6134:AF	UKB-b-6134:SS	UKB-b-6134:EZ	UKB-b-6134:SI	UKB-b-6134:NC	UKB-b-6134:ID

1 :	 1	49298	rs10399793	T	C	.	PASS	0.623238	0.0017795	0.00366743	0.200659	0.623238	.	.	.	.	rs10399793



In [2]:
# debug cell
import gzip
with gzip.open('./ukb-b-6134.vcf.gz') as f:
    print(f)

FileNotFoundError: [Errno 2] No such file or directory: './ukb-b-6134.vcf.gz'

In [3]:
import os
import json
import pickle
import time
import gzip
import shutil

def generate_gwas_output_as_tsv_file(inVCF_File, writeFile=None):
    root = './tabular_format_gwas_data/'
    t0 = time.time()
    if writeFile is None:
        writeFile = root + inVCF_File.split('/')[-1].split('.')[0] + '.tsv'
    
    print("Processing:", inVCF_File)
    if ".gz" == inVCF_File[-3:]:
        with gzip.open(inVCF_File, 'rb') as f_in:
            with open(root+'temp_vcf.vcf', 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
        inVCF_File = root+'temp_vcf.vcf'
    """
    Convert all the VCF into simplified tsv format
    Source: https://github.com/everestial/VCF-simplify#table-of-contents
    python3 ./VCF-Simplify/VCF-Simplify/VcfSimplify.py SimplifyVCF -toType table -inVCF ./ukb-b-6134.vcf -outFile ./simple_table.txt
    """
    vcf_simplify_path = "./VCF-Simplify/VCF-Simplify/VcfSimplify.py"
    out_File = root+'temp_table.tsv'
    os.system(f"python3 {vcf_simplify_path} SimplifyVCF -toType table -inVCF {inVCF_File} -outFile {out_File}")
        
    f_read = open(out_File, 'r')
    f_write = open(writeFile, 'w')
    cnt = 0
    for line in f_read:
    #     print(line)
        newliner = ''
        if '\n' == line[-1]:
            newliner = '\n'
            line = line.strip()
        if 'CHROM' in line:
            line = line.upper() + '\tPVAL_generated_from_LP'
        else:
            LP = float(line.split('\t')[10])
            p_val = str(10**(-1 * LP))
            line = line + '\t' + p_val
        line = line + newliner
    #     print(line)
        f_write.write(line)
        cnt += 1
    #     if cnt == 20: break

    f_read.close()
    f_write.close()
    print(f'Total SNPs: {cnt} | Total exec time: {time.time() - t0} seconds')

In [ ]:
# generate_gwas_output_as_tsv_file(inVCF_File='./ukb-b-6134.vcf.gz', writeFile=None)

In [ ]:

# ' abc def\n'.strip()

In [ ]:
# 10**(-1*0.200659) #0.6300007

In [4]:
# %pip install openpyxl
import pandas as pd

category_map = {
    'Continuous' : 'beta', 
    'Categorical Ordered (assumed continuous)': 'beta',
    'Binary': 'or',
    'NA (Possibly binary)': 'or'
}

df = pd.read_excel('GWAS_datasets_to_consider.xlsx', engine='openpyxl')[['GWAS_ID', 'Trait', 'Category']]
print(df['Category'].unique())
GWAS_ID = 'ieu-b-111'
category = category_map[df[df['GWAS_ID'] == GWAS_ID].to_numpy()[0, 2]] 
category

['Continuous' 'Binary' 'Risk factor' 'NA (Possibly binary)'
 'Categorical Ordered (assumed continuous)']


'beta'

In [5]:
from glob import glob
# all_gwas_vcf_files = glob('/mnt/c/Users/HP/Downloads/Compressed/GWAS/*')
# ROOT = '/mnt/c/Users/HP/Downloads/Compressed/GWAS/'
ALL_GWAS_IDS = list(set(df['GWAS_ID'].to_numpy().tolist()) - {'bbj-a-46', 'bbj-a-78', 'ieu-a-1283', 'ukb-a-257','ukb-b-2209','ukb-b-17627','ieub-109','ieu-b-110','ieu-b-111','met-d-Total_C'})
base_files = {}
for GWAS_ID in ALL_GWAS_IDS:
    print(GWAS_ID)
#     file = ROOT + GWAS_ID + '.vcf.gz'
    base_files[GWAS_ID] = category_map[df[df['GWAS_ID'] == GWAS_ID].to_numpy()[0, 2]] 
#     generate_gwas_output_as_tsv_file(inVCF_File=file, writeFile=None)

ukb-b-770
ieu-b-38
ukb-d-20405_1
ukb-d-20405_0
ieu-b-109
ieu-b-39
ukb-b-13806
ukb-b-19953
ukb-b-4424
ieu-b-25
ukb-b-18275
ukb-b-12064
ukb-b-7663
ukb-b-3957
ukb-b-8476
ukb-b-6134
ukb-d-20405_2
ukb-b-6324


In [6]:
# base_files
for GWAS_ID in base_files:
    print(GWAS_ID)
    

ukb-b-770
ieu-b-38
ukb-d-20405_1
ukb-d-20405_0
ieu-b-109
ieu-b-39
ukb-b-13806
ukb-b-19953
ukb-b-4424
ieu-b-25
ukb-b-18275
ukb-b-12064
ukb-b-7663
ukb-b-3957
ukb-b-8476
ukb-b-6134
ukb-d-20405_2
ukb-b-6324


In [7]:
gender_map = {'Female': 0,'Male': 1}
ADNIMERGE = pd.read_csv('ADNIMERGE.csv')
def get_gender_and_age(PTID):
    gender = ADNIMERGE[ADNIMERGE['PTID'] == PTID]['PTGENDER'].dropna().unique()[0]
    age = ADNIMERGE[ADNIMERGE['PTID'] == PTID]['AGE'].dropna().unique()[0]
    return str(gender_map[gender]) + ' ' + str(age)

# NUM_TRAINING_SAMPLES = int(830 * .7)
# print('NUM_TRAINING_SAMPLES:', NUM_TRAINING_SAMPLES)
# f_writable = open('./COVAR_FILE.txt', 'w')
# f_TRAINING_SAMPLES = open('./TRAINING_SAMPLES.txt', 'w')
# with open('/mnt/c/Users/HP/Documents/GWA_tutorial/2_Population_stratification/covar_mds.txt') as f:
#     for i, line in enumerate(f):
#         if 'FID IID' in line:
#             line = line[:-1] + ' GENDER AGE\n'
#         else:
#             line = line[:-1] + ' ' + get_gender_and_age(PTID=line.split(' ')[1]) + '\n'
#         if i < NUM_TRAINING_SAMPLES:
#             f_TRAINING_SAMPLES.write(' '.join(line.split(' ')[:2])+'\n')
#         f_writable.write(line)
# #         print(line)
# f_writable.close()

/tmp/ipykernel_23164/1785225935.py:2: DtypeWarning: Columns (18,19,20,103,104) have mixed types. Specify dtype option on import or set low_memory=False.
  ADNIMERGE = pd.read_csv('ADNIMERGE.csv')


In [12]:
def run_prsice(GWAS_ID, redo=False):
#     TARGET_DATA = '/mnt/c/Users/HP/Documents/GWA_tutorial/2_Population_stratification/ADNI' # old and smaller dataset
    TARGET_DATA = '../larger_dataset/larger_dataset' # new and larger dataset
    '''
    return_status --> 0: success | 256: Error. Execusion Halted | 2: 
    '''
    if os.path.isfile(f'./PRSice_output/{GWAS_ID}/'):
        return_status = os.system(f'mkdir ./PRSice_output/{GWAS_ID}/')
        print('0:', return_status)
        if return_status == 256 and not redo:
            return return_status
    print('stat',{base_files[GWAS_ID]})
    prsice_command = f'Rscript ./PRSice_linux/PRSice.R \
        --dir ./PRSice_output \
        --prsice ./PRSice_linux/PRSice_linux \
        --base ./tabular_format_gwas_data/{GWAS_ID}.tsv \
        --target {TARGET_DATA} \
        --thread 1 \
        --stat {base_files[GWAS_ID].upper()} \
        --{base_files[GWAS_ID]} \
        --binary-target F \
        --quantile 10 \
        --out ./PRSice_output/{GWAS_ID}/{GWAS_ID} \
        --snp ID --chr CHROM --bp POS --A2 REF --A1 ALT --stat {GWAS_ID.upper()}:ES --pvalue PVAL_generated_from_LP \
        --score std \
        --cov ../larger_dataset/COVAR_FILE_bigger_dataset.txt \
        --extract ./PRSice_output/{GWAS_ID}/{GWAS_ID}.valid'

#     return_status = os.system(prsice_command + '  --keep ./TRAINING_SAMPLES.txt') # generate the file named {GWAS_ID}.valid
#     return_status = os.system(prsice_command) # generate the file named {GWAS_ID}.valid
#     print('1a:', return_status)
#     return
#     if os.path.isfile(f'./PRSice_output/{GWAS_ID}/{GWAS_ID}.valid'): 
#         prsice_command += f' --extract ./PRSice_output/{GWAS_ID}/{GWAS_ID}.valid' 
#         return_status = os.system(prsice_command + '  --keep ./TRAINING_SAMPLES.txt') # perform the regression analysis on the training samples only and choose the best p-value-threshold
#         return_status = os.system(prsice_command) # perform the regression analysis on the training samples only and choose the best p-value-threshold
#         print('1b:', return_status)
#         if return_status != 0:
# #             return_status = os.system(prsice_command) # perform the regression analysis on the training samples only and choose the best p-value-threshold
#             print('1c:', return_status)
#     return_status = os.system(prsice_command + '  --all-score --no-regress') # Do not perform the regression analysis and simply output all PRS.
#     return_status = os.system(prsice_command + '  --all-score') # Do not perform the regression analysis and simply output all PRS.
#     print('2:', return_status)
    # in later steps, select the PRS values for the selected p-value-threshold for all the samples (both training and testing)
    print(prsice_command)
    print('stat',{base_files[GWAS_ID]})
    return_status = os.system(prsice_command) # generate the file named {GWAS_ID}.valid
    print('1:', return_status)
    return_status = os.system('echo "===== Done ====="')
    print('2:', return_status) 
#     os.system(f'mv ./prsice_output/{GWAS_ID}.* ./prsice_output/{GWAS_ID}/')
# https://github.com/adiamb/Remove-duplicate-snps-plink

In [13]:
for GWAS_ID in base_files:
    if GWAS_ID == 'ukb-b-6134':
        print('*****GWAS ID IS*****',GWAS_ID)
        run_prsice(GWAS_ID=GWAS_ID)
        break

*****GWAS ID IS***** ukb-b-6134
stat {'beta'}
Rscript ./PRSice_linux/PRSice.R         --dir ./PRSice_output         --prsice ./PRSice_linux/PRSice_linux         --base ./tabular_format_gwas_data/ukb-b-6134.tsv         --target ../larger_dataset/larger_dataset         --thread 1         --stat BETA         --beta         --binary-target F         --quantile 10         --out ./PRSice_output/ukb-b-6134/ukb-b-6134         --snp ID --chr CHROM --bp POS --A2 REF --A1 ALT --stat UKB-B-6134:ES --pvalue PVAL_generated_from_LP         --score std         --cov ../larger_dataset/COVAR_FILE_bigger_dataset.txt         --extract ./PRSice_output/ukb-b-6134/ukb-b-6134.valid
stat {'beta'}


PRSice 2.3.5 (2021-09-20) 
https://github.com/choishingwan/PRSice
(C) 2016-2020 Shing Wan (Sam) Choi and Paul F. O'Reilly
GNU General Public License v3
If you use PRSice in any published work, please cite:
Choi SW, O'Reilly PF.
PRSice-2: Polygenic Risk Score Software for Biobank-Scale Data.
GigaScience 8, no. 7 (July 1, 2019)
2022-07-05 06:40:31
./PRSice_linux/PRSice_linux \
    --a1 ALT \
    --a2 REF \
    --bar-levels 0.001,0.05,0.1,0.2,0.3,0.4,0.5,1 \
    --base ./tabular_format_gwas_data/ukb-b-6134.tsv \
    --beta  \
    --binary-target F \
    --bp POS \
    --chr CHROM \
    --clump-kb 250kb \
    --clump-p 1.000000 \
    --clump-r2 0.100000 \
    --cov ../larger_dataset/COVAR_FILE_bigger_dataset.txt \
    --extract ./PRSice_output/ukb-b-6134/ukb-b-6134.valid \
    --interval 5e-05 \
    --lower 5e-08 \
    --num-auto 22 \
    --out ./PRSice_output/ukb-b-6134/ukb-b-6134 \
    --pvalue PVAL_generated_from_LP \
    --score std \
    --seed 1736013754 \
    --snp ID \
    --stat U

Begin plotting
Current Rscript version = 2.3.3
Plotting the quantile plot
Plotting Bar Plot
Plotting the high resolution plot
1: 0
===== Done =====
2: 0


In [ ]:
# run_prsice(GWAS_ID='ukb-b-17627')

In [14]:
!ls 'tabular_format_gwas_data/{GWAS_ID}.tsv'

tabular_format_gwas_data/ukb-b-6134.tsv


In [15]:
base_files
GWAS_ID = 'ukb-b-18275'

#====

def get_prs_values(GWAS_ID):
    prsice_output = f'./PRSice_output/{GWAS_ID}/'

    if False:
        with open(prsice_output+f'{GWAS_ID}.all_score', 'r') as f:
            lines = f.readlines()
            lines = ['\t'.join(x.split()) for x in lines]
            lines = '\n'.join(lines)

        with open(prsice_output+f'{GWAS_ID}.all_score.tsv', 'w') as f:
            f.write(lines)

        # best_p_val_threshold = '0.00025005'
        best_p_val_threshold = str(open(prsice_output+f'{GWAS_ID}.summary').readlines()[1].split('\t')[2]) 
    #     print(best_p_val_threshold) 
    prs_array = pd.read_csv(prsice_output+f'{GWAS_ID}.best', ' ')['PRS'].to_numpy() 
    return prs_array

In [16]:
get_prs_values(GWAS_ID=GWAS_ID).shape[0]

/tmp/ipykernel_23164/1595037025.py:1: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  get_prs_values(GWAS_ID=GWAS_ID).shape[0]


1816

In [17]:
import numpy as np
import os.path

PRS_feature_matrix = np.zeros([len(base_files), 1816])
for i, GWAS_ID in enumerate(base_files):
    print(GWAS_ID)
    prsice_output = f'./PRSice_output/{GWAS_ID}/{GWAS_ID}.best'
    if os.path.exists(prsice_output):  
        PRS_feature_matrix[i, :] = get_prs_values(GWAS_ID=GWAS_ID)
    

ukb-b-770
ieu-b-38
ukb-d-20405_1
ukb-d-20405_0


/tmp/ipykernel_23164/3576494164.py:9: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  PRS_feature_matrix[i, :] = get_prs_values(GWAS_ID=GWAS_ID)
/tmp/ipykernel_23164/3576494164.py:9: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  PRS_feature_matrix[i, :] = get_prs_values(GWAS_ID=GWAS_ID)
/tmp/ipykernel_23164/3576494164.py:9: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  PRS_feature_matrix[i, :] = get_prs_values(GWAS_ID=GWAS_ID)
/tmp/ipykernel_23164/3576494164.py:9: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  PRS_feature_matrix[i, :] = get_prs_values(GWAS_ID=GWAS_ID)


ieu-b-109
ieu-b-39
ukb-b-13806
ukb-b-19953


/tmp/ipykernel_23164/3576494164.py:9: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  PRS_feature_matrix[i, :] = get_prs_values(GWAS_ID=GWAS_ID)
/tmp/ipykernel_23164/3576494164.py:9: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  PRS_feature_matrix[i, :] = get_prs_values(GWAS_ID=GWAS_ID)
/tmp/ipykernel_23164/3576494164.py:9: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  PRS_feature_matrix[i, :] = get_prs_values(GWAS_ID=GWAS_ID)
/tmp/ipykernel_23164/3576494164.py:9: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  PRS_feature_matrix[i, :] = get_prs_values(GWAS_ID=GWAS_ID)


ukb-b-4424
ieu-b-25
ukb-b-18275
ukb-b-12064
ukb-b-7663


/tmp/ipykernel_23164/3576494164.py:9: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  PRS_feature_matrix[i, :] = get_prs_values(GWAS_ID=GWAS_ID)
/tmp/ipykernel_23164/3576494164.py:9: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  PRS_feature_matrix[i, :] = get_prs_values(GWAS_ID=GWAS_ID)
/tmp/ipykernel_23164/3576494164.py:9: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  PRS_feature_matrix[i, :] = get_prs_values(GWAS_ID=GWAS_ID)
/tmp/ipykernel_23164/3576494164.py:9: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  PRS_feature_matrix[i, :] = get_prs_values(GWAS_ID=GWAS_ID)
/tmp/ipykernel_23164/3576494164.py:9: FutureWarning:

ukb-b-3957
ukb-b-8476
ukb-b-6134
ukb-d-20405_2
ukb-b-6324


/tmp/ipykernel_23164/3576494164.py:9: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  PRS_feature_matrix[i, :] = get_prs_values(GWAS_ID=GWAS_ID)
/tmp/ipykernel_23164/3576494164.py:9: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  PRS_feature_matrix[i, :] = get_prs_values(GWAS_ID=GWAS_ID)
/tmp/ipykernel_23164/3576494164.py:9: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  PRS_feature_matrix[i, :] = get_prs_values(GWAS_ID=GWAS_ID)
/tmp/ipykernel_23164/3576494164.py:9: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  PRS_feature_matrix[i, :] = get_prs_values(GWAS_ID=GWAS_ID)
/tmp/ipykernel_23164/3576494164.py:9: FutureWarning:

In [18]:
PRS_feature_matrix = PRS_feature_matrix.T
np.save('PRS_feature_matrix', PRS_feature_matrix)

In [19]:
PRS_feature_matrix.shape
import random

In [20]:
TRAIN_SAMPLE_SIZE = 550
train_samples = list(range(847))
random.shuffle(train_samples)
train_samples, test_samples = train_samples[:TRAIN_SAMPLE_SIZE], train_samples[TRAIN_SAMPLE_SIZE:]
print(test_samples)
print(len(train_samples), len(test_samples), len(train_samples) + len(test_samples))
PRS_feature_matrix[train_samples].mean(0), PRS_feature_matrix[train_samples].std(0)

[818, 222, 409, 300, 664, 370, 396, 608, 249, 374, 714, 97, 233, 565, 503, 192, 524, 688, 751, 212, 624, 327, 241, 767, 804, 94, 226, 763, 700, 749, 718, 773, 358, 210, 639, 114, 625, 104, 482, 545, 681, 830, 400, 196, 124, 244, 459, 355, 648, 313, 705, 546, 70, 597, 98, 807, 795, 559, 711, 579, 319, 461, 520, 757, 526, 314, 406, 134, 211, 563, 199, 399, 11, 485, 410, 685, 771, 385, 493, 144, 379, 41, 228, 188, 817, 242, 617, 132, 36, 136, 219, 842, 229, 382, 194, 699, 270, 684, 766, 621, 240, 577, 636, 43, 764, 537, 398, 419, 555, 202, 477, 380, 33, 759, 668, 326, 348, 781, 809, 145, 139, 390, 709, 272, 403, 9, 529, 113, 215, 350, 564, 615, 107, 822, 155, 38, 556, 335, 40, 629, 266, 815, 121, 247, 238, 642, 611, 600, 372, 496, 397, 446, 122, 449, 235, 84, 297, 207, 110, 785, 812, 578, 465, 149, 423, 651, 505, 514, 422, 102, 846, 13, 487, 86, 88, 417, 352, 534, 365, 279, 833, 616, 729, 758, 673, 49, 198, 320, 32, 531, 770, 127, 59, 474, 281, 338, 0, 125, 151, 748, 557, 12, 80, 223, 678

(array([ 0.05308238,  0.03451169, -0.21770446, -0.51659621, -0.01568063,
         0.02917379, -0.06685639,  0.03581486,  0.12414165, -0.02429693,
        -0.57283057, -0.10043958, -0.5589993 ,  0.02353178, -0.64111237,
         0.03097957, -0.49644052,  0.0326984 ]),
 array([1.02887107, 0.98538162, 0.85797973, 0.58797811, 0.98258345,
        1.00093132, 0.98533474, 1.01065323, 0.68248991, 0.92532206,
        0.59265129, 0.94301748, 0.56555325, 0.94862211, 0.71645114,
        0.98800414, 0.53086297, 0.9527379 ]))

In [21]:
PRS_feature_matrix[test_samples].mean(0), PRS_feature_matrix[test_samples].std(0)

(array([-0.01081337, -0.01227618, -0.3021462 , -0.52705209,  0.13395638,
        -0.05958485, -0.09604769, -0.08230944,  0.12333936,  0.04324866,
        -0.60816329, -0.31198319, -0.58261849, -0.00807834, -0.72856645,
        -0.06363957, -0.49800506, -0.0407727 ]),
 array([0.91144598, 0.94008246, 0.8827431 , 0.5577856 , 1.02065063,
        0.96791466, 0.97346989, 0.99592809, 0.67461437, 1.03599949,
        0.55956999, 0.92293673, 0.53275235, 0.95155192, 0.66512051,
        1.02727449, 0.49098899, 1.05159588]))

In [ ]:
PRS_feature_matrix[:].mean(0), PRS_feature_matrix[:].std(0)

In [ ]:
import json
import pandas as pd
GWAS_ID = 'ieu-b-25'
usable_samples_ADNI = pd.read_csv(f'./PRSice_output/{GWAS_ID}/{GWAS_ID}.best', ' ')['IID'].to_numpy().tolist()
usable_samples_ADNI = {'_'.join(sample.split('_')[1:]):idx for idx,sample in enumerate(usable_samples_ADNI)}
# json.dump(usable_samples_ADNI, open('./usable_samples_ADNI.json', 'w'))
type(usable_samples_ADNI), len(usable_samples_ADNI)

In [ ]:
!ls prsice_output

In [ ]:
import json
import numpy as np
PRS_feature_matrix = np.load('./PRS_feature_matrix.npy')
# usable_samples_ADNI = json.load(open('./usable_samples_ADNI.json'))
print(PRS_feature_matrix.shape, usable_samples_ADNI.__len__())

usable_samples_ADNI

In [ ]:
THRESHOLD_MONTH = 12*2
ADNIMERGE = pd.read_csv('ADNIMERGE.csv')
ADNI_TEMP = ADNIMERGE[['RID', 'PTID', 'DX', 'Month']].dropna().sort_values(by=['PTID', 'Month'])
Dementia = ADNI_TEMP[ADNI_TEMP['DX']=='Dementia']
Samples_Dementia = set(Dementia['PTID'].unique())
Samples_NonDementia = set(ADNI_TEMP['PTID'].unique()) - Samples_Dementia
# Samples_NonDementia
Final_Samples_Dementia = set()
for sample in Samples_Dementia:
    last_dx = ADNI_TEMP[ADNI_TEMP['PTID']==sample]['DX'].iloc[-1]
    last_month_of_dx = ADNI_TEMP[ADNI_TEMP['PTID']==sample]['Month'].iloc[-1]
    if last_month_of_dx >= THRESHOLD_MONTH or True:
        if last_dx == 'Dementia' and sample in usable_samples_ADNI:
                Final_Samples_Dementia = Final_Samples_Dementia.union({sample})

Final_Samples_NonDementia = set()
for sample in Samples_NonDementia:
    last_month_of_dx = ADNI_TEMP[ADNI_TEMP['PTID']==sample]['Month'].iloc[-1]
    if last_month_of_dx >= THRESHOLD_MONTH and sample in usable_samples_ADNI:
        Final_Samples_NonDementia = Final_Samples_NonDementia | {sample} # union symbol in python sets
    
# sum(ADNI_TEMP['Month'] == 6), ADNI_TEMP.shape
# Dementia
Final_Samples_Dementia = [[x, 1] for x in Final_Samples_Dementia]
Final_Samples_NonDementia = [[x, 0] for x in Final_Samples_NonDementia]

Final_Samples = Final_Samples_Dementia + Final_Samples_NonDementia
json.dump(Final_Samples, open('Final_Samples.json', 'w'))

Final_Samples_Dementia.__len__(), Final_Samples_NonDementia.__len__(), Final_Samples.__len__()

In [ ]:
import json
import pandas as pd
GWAS_ID = 'ieu-b-25'
usable_samples_ADNI = pd.read_csv(f'./PRSice_output/{GWAS_ID}/{GWAS_ID}.best', ' ')['IID'].to_numpy().tolist()
usable_samples_ADNI = {sample:idx for idx,sample in enumerate(usable_samples_ADNI)}
json.dump(usable_samples_ADNI, open('./usable_samples_ADNI.json', 'w'))

In [ ]:
ALL_GWAS_IDS = [key for key in base_files]
print(ALL_GWAS_IDS)

In [ ]:
df

In [ ]:
df = pd.read_excel('GWAS_datasets_to_consider.xlsx', engine='openpyxl')[['GWAS_ID', 'Trait', 'Category']]
all_traits = []
for GWAS_ID in ALL_GWAS_IDS:
    trait = str(df[df['GWAS_ID'] == GWAS_ID]['Trait'].to_numpy()[0])
    print(GWAS_ID, ':', trait)
    all_traits += [trait]
print(all_traits)

In [ ]:
import torch
torch.__version__

In [ ]:
# this is for demo purpose
prsice_command = f'Rscript ./PRSice_linux/PRSice.R --dir ./PRSice_linux	\
--prsice ./PRSice_linux/PRSice_linux     \
--base ./PRSice_linux/simple_table_with_pval.txt     \
--target ./PRSice_linux/ADNI_plink     \
--thread 1     \
--beta     \
--binary-target F	\
--out ./PRSice_output/PRSice	\
--snp ID --chr CHROM --bp POS --A2 REF --A1 ALT --stat UKB-b-6134:ES --pvalue PVAL_generated_from_LP	\
--extract ./PRSice_output/PRSice.valid'

In [ ]:
    # this is for demo purpose
    import os
    return_status = os.system(prsice_command) # generate the file named {GWAS_ID}.valid
    print('1a:', return_status)